这个文件重新做的是 山东、黑龙江、青海、内蒙古 几个省的地图匹配

In [25]:
import pandas as pd

# 文件路径
file_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120地图匹配_v6.xlsx'
df = pd.read_excel(file_path)
df = df.iloc[17264:]
# 文件路径
file_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/国标码/county1953.xls'
# 读取 .xls 文件
df2 = pd.read_excel(file_path)
df2.head()

,PROVGB,CITYGB,CNTYGB,CPROV,EPROV,CCNTY,ECNTY,A53001,A53002
0,71,7100,710000,台湾,Taiwan,台湾,Taiwan,0.0,0.0
1,44,4460,446020,广东,Guangdong,珠海县,Zhuhai Xian,27523.0,126146.0
2,44,4490,449039,广东,Guangdong,南沙群岛,Nansha Qun Dao,0.0,0.0
3,44,4490,449037,广东,Guangdong,西沙群岛,Xisha Qun Dao,0.0,0.0
4,44,4490,449038,广东,Guangdong,中沙群岛,Zhongsha Qun Dao,0.0,0.0


模糊匹配，可以给freelancer来检查的东西

In [26]:
import pandas as pd
from fuzzywuzzy import process

# 假设 df 和 df2 是你的两个 DataFrame
# df 包含 'name' 和 'area' 列
# df2 包含 'CPROV', 'CCNTY', 'PROVGB', 'CITYGB', 'CNTYGB' 等列

# 1. 将 df2 的 CPROV 列和 CCNTY 列字符拼接到一块，新生成一列 AREA
df2['AREA'] = df2['CPROV'].astype(str) + df2['CCNTY'].astype(str)

# 2. 在 df2 所有列名后都加上 "_1953"
df2.columns = [col + '_1953' for col in df2.columns]

# 3. 定义匹配逻辑
def find_best_match(row_area, df2, scorer):
    # 提取 area 的前两个字符（假设 CPROV_1953 是两位编码）
    area_prov = row_area[:2]
    
    # 过滤 df2，只保留 CPROV_1953 与 area_prov 完全匹配的行
    filtered_df2 = df2[df2['CPROV_1953'].astype(str) == area_prov]
    
    if filtered_df2.empty:
        return None, None  # 如果没有匹配项，返回 None
    
    # 提取 area 的剩余部分
    area_remainder = row_area[2:]
    
    # 对 filtered_df2 中的 CCNTY_1953 进行相似度匹配
    best_match = None
    best_score = 0
    for _, row in filtered_df2.iterrows():
        score = scorer(area_remainder, str(row['CCNTY_1953']))
        if score > best_score:
            best_score = score
            best_match = row
    
    return best_match, best_score

# 创建一个新的 DataFrame 来存储结果
result_df = df[['name', 'area']].copy()

# 添加 df2 的列到 result_df
for col in ['PROVGB_1953', 'CITYGB_1953', 'CNTYGB_1953', 'CPROV_1953', 'CCNTY_1953']:
    result_df[col] = None

# 添加一列用于存储相似度得分
result_df['similarity_score'] = None

# 遍历 df 的每一行，找到 df2 中相似度最高的行
for index, row in df.iterrows():
    best_match, similarity_score = find_best_match(row['area'], df2, process.fuzz.ratio)
    if best_match is not None and similarity_score >= 60:  # 仅当相似度得分 >= 60 时填充数据
        result_df.at[index, 'PROVGB_1953'] = best_match['PROVGB_1953']
        result_df.at[index, 'CITYGB_1953'] = best_match['CITYGB_1953']
        result_df.at[index, 'CNTYGB_1953'] = best_match['CNTYGB_1953']
        result_df.at[index, 'CPROV_1953'] = best_match['CPROV_1953']
        result_df.at[index, 'CCNTY_1953'] = best_match['CCNTY_1953']
        result_df.at[index, 'similarity_score'] = similarity_score  # 存储相似度得分
    else:
        # 如果相似度得分 < 60，将所有相关列设置为空值
        result_df.at[index, 'PROVGB_1953'] = None
        result_df.at[index, 'CITYGB_1953'] = None
        result_df.at[index, 'CNTYGB_1953'] = None
        result_df.at[index, 'CPROV_1953'] = None
        result_df.at[index, 'CCNTY_1953'] = None
        result_df.at[index, 'similarity_score'] = None

result_df.head()

,name,area,PROVGB_1953,CITYGB_1953,CNTYGB_1953,CPROV_1953,CCNTY_1953,similarity_score
17264,17264__002日月运行,山东东明县,37,3702,370240,山东,即东县,67
17265,17265__003女蜗补天,山东济南市,37,3701,370100,山东,济南市,100
17266,17266__004兄妹补天出彩虹,山东东营市,None,None,None,None,None,None
17267,17267__005嫦娥奔月,山东济南市,37,3701,370100,山东,济南市,100
17268,17268__006亚拉射月,山东临沐县,37,3707,370724,山东,临朐县,67


In [27]:
result_df.to_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120area_match_add_province_1953.csv', index=False)

In [92]:
import pandas as pd

# 文件路径
file_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120地图匹配_v6.xlsx'
df = pd.read_excel(file_path)
df = df.iloc[17264:]
# 文件路径
file_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/国标码/county2000.xls'
# 读取 .xls 文件
df2 = pd.read_excel(file_path)
df2.head()

,AREA,PERIMETER,PROVGB,CITYGB,CNTYGB,CPROV,EPROV,CCITY,ECITY,CCNTY,...,L2000220,L2000221,L2000222,L2000223,L2000224,L2000225,L2000226,L2000227,L2000228,L2000229
0,0.007,0.377,33,3306,330602,浙江,Zhejiang,绍兴市,Shaoxing,越城区,...,16972,3270,10331,9911,12336,7906,10219,6086,2469,1468
1,0.009,0.478,33,3306,330621,浙江,Zhejiang,绍兴市,Shaoxing,绍兴县,...,19139,5226,9086,15279,9772,14593,18951,1935,2044,1435
2,0.003,0.260,11,1101,110101,北京,Beijing,北京市辖区,Beijing (Districts),东城区,...,16139,41,7574,8606,7129,9051,1410,4475,10027,268
3,0.003,0.246,11,1101,110102,北京,Beijing,北京市辖区,Beijing (Districts),西城区,...,22607,32,12923,9716,14319,8320,1746,8827,11491,575
4,0.002,0.233,11,1101,110103,北京,Beijing,北京市辖区,Beijing (Districts),崇文区,...,11640,51,4195,7496,4946,6745,885,2433,7052,1321


In [93]:
import pandas as pd
from fuzzywuzzy import process

# 假设 df 和 df2 是你的两个 DataFrame
# df 包含 'name' 和 'area' 列
# df2 包含 'CPROV', 'CCNTY', 'PROVGB', 'CITYGB', 'CNTYGB' 等列

# 定义后缀变量
suffix = '_2000'  # 可以修改为其他后缀，例如 '_2023'

# 1. 将 df2 的 CPROV 列和 CCNTY 列字符拼接到一块，新生成一列 AREA
df2['AREA'] = df2['CPROV'].astype(str) + df2['CCNTY'].astype(str)

# 2. 在 df2 所有列名后都加上后缀变量
df2.columns = [col + suffix for col in df2.columns]

# 3. 定义匹配逻辑
def find_best_match(row_area, df2, scorer, suffix):
    # 提取 area 的前两位或前三位字符（假设 CPROV 是两位或三位编码）
    # 先尝试匹配前两位，如果失败则尝试匹配前三位
    area_prov_2 = row_area[:2]  # 前两位
    area_prov_3 = row_area[:3]  # 前三位
    
    # 过滤 df2，先尝试匹配前两位
    filtered_df2 = df2[df2[f'CPROV{suffix}'].astype(str) == area_prov_2]
    
    # 如果前两位没有匹配到，尝试匹配前三位
    if filtered_df2.empty:
        filtered_df2 = df2[df2[f'CPROV{suffix}'].astype(str) == area_prov_3]
    
    if filtered_df2.empty:
        return None, None  # 如果没有匹配项，返回 None
    
    # 提取 area 的剩余部分
    area_remainder = row_area[len(filtered_df2.iloc[0][f'CPROV{suffix}']):]
    
    # 对 filtered_df2 中的 CCNTY + suffix 进行相似度匹配
    best_match = None
    best_score = 0
    for _, row in filtered_df2.iterrows():
        score = scorer(area_remainder, str(row[f'CCNTY{suffix}']))
        if score > best_score:
            best_score = score
            best_match = row
    
    return best_match, best_score

# 创建一个新的 DataFrame 来存储结果
result_df = df[['name', 'area']].copy()

# 添加 df2 的列到 result_df
for col in ['PROVGB', 'CITYGB', 'CNTYGB', 'CPROV', 'CCNTY']:
    result_df[col + suffix] = None

# 添加一列用于存储相似度得分
result_df['similarity_score'] = None

# 遍历 df 的每一行，找到 df2 中相似度最高的行
for index, row in df.iterrows():
    best_match, similarity_score = find_best_match(row['area'], df2, process.fuzz.ratio, suffix)
    if best_match is not None and similarity_score >= 0:  # 仅当相似度得分 >= 60 时填充数据
        result_df.at[index, f'PROVGB{suffix}'] = best_match[f'PROVGB{suffix}']
        result_df.at[index, f'CITYGB{suffix}'] = best_match[f'CITYGB{suffix}']
        result_df.at[index, f'CNTYGB{suffix}'] = best_match[f'CNTYGB{suffix}']
        result_df.at[index, f'CPROV{suffix}'] = best_match[f'CPROV{suffix}']
        result_df.at[index, f'CCNTY{suffix}'] = best_match[f'CCNTY{suffix}']
        result_df.at[index, 'similarity_score'] = similarity_score  # 存储相似度得分
    else:
        # 如果相似度得分 < 60，将所有相关列设置为空值
        result_df.at[index, f'PROVGB{suffix}'] = None
        result_df.at[index, f'CITYGB{suffix}'] = None
        result_df.at[index, f'CNTYGB{suffix}'] = None
        result_df.at[index, f'CPROV{suffix}'] = None
        result_df.at[index, f'CCNTY{suffix}'] = None
        result_df.at[index, 'similarity_score'] = None

In [94]:
result_df.to_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120area_match_add_province_2000.csv', index=False)

得分是100的就匹配到地图里去

In [ ]:
import pandas as pd

# 定义变量
year = '2000'  # 可以修改为其他年份

# 读取Excel文件
excel_file_path = f'/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120地图匹配_v6.xlsx'
df_excel = pd.read_excel(excel_file_path)

# 读取CSV文件
csv_file_path = f'/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120area_match_add_province/250120area_match_add_province_{year}.csv'
df_csv = pd.read_csv(csv_file_path)

# 1. 按照 'name' 列合并两个表，并指定后缀
merged_df = pd.merge(df_excel, df_csv, on='name', how='left', suffixes=('', '_csv'))

# 检查合并后的列名
print("合并后的列名:", merged_df.columns.tolist())

# 2. 根据条件更新 df_excel 的列
# 如果 similarity_score 为 100，则将 PROVGB_{year}_csv, CITYGB_{year}_csv, CNTYGB_{year}_csv 的值更新到 df_excel 中
condition = merged_df['similarity_score'] >= 67

# 更新 df_excel 的列
for col in ['PROVGB', 'CITYGB', 'CNTYGB']:
    col_year = f'{col}_{year}'  # 例如 PROVGB_1964
    col_csv = f'{col_year}_csv'  # 例如 PROVGB_1964_csv

    if col_csv in merged_df.columns:  # 确保列存在
        df_excel.loc[condition, col_year] = merged_df.loc[condition, col_csv]

# 保存更新后的 df_excel 到新的 Excel 文件
output_excel_path = f'/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120地图匹配_v6_2000.xlsx'
df_excel.to_excel(output_excel_path, index=False)

print(f"更新后的数据已保存到: {output_excel_path}")

合并后的列名: ['rowid', 'name', 'cailu', 'provgb', 'citygb', 'cntygb', 'area', 'geocode', 'PROVGB_1953', 'CITYGB_1953', 'CNTYGB_1953', 'PROVGB_1964', 'CITYGB_1964', 'CNTYGB_1964', 'PROVGB_1982', 'CITYGB_1982', 'CNTYGB_1982', 'PROVGB_1990a', 'CITYGB_1990a', 'CNTYGB_1990a', 'PROVGB_2000', 'CITYGB_2000', 'CNTYGB_2000', 'PROVGB_2010', 'CITYGB_2010', 'CNTYGB_2010', 'new_name', 'area_group', 'area_csv', 'PROVGB_2000_csv', 'CITYGB_2000_csv', 'CNTYGB_2000_csv', 'CPROV_2000', 'CCNTY_2000', 'similarity_score']
更新后的数据已保存到: /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120地图匹配_v6_2000.xlsx
